### Import the libraries

In [1]:
import boto3
import pandas as pd
import awswrangler as wr
from io import StringIO
from sagemaker import image_uris, TrainingInput
from sagemaker.estimator import Estimator
import datetime
import boto3
import awswrangler as wr
import pandas as pd
import sagemaker
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.model_monitor import CronExpressionGenerator
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor.model_monitoring import EndpointInput
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from io import StringIO
import datetime
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
%store

Stored variables and their in-db values:
account_id                                -> '453322325373'
boto_session                              -> '<unavailable>'
bucket_name                               -> 'housing-dataset-3320'
df_test_saved_to_s3                       -> 's3://housing-dataset-8749/processed/housing/test.
df_train_saved_to_s3                      -> 's3://housing-dataset-8749/processed/housing/train
df_validation_saved_to_s3                 -> 's3://housing-dataset-8749/processed/housing/valid
estimator                                 -> '<unavailable>'
feature_store_session                     -> '<unavailable>'
featurestore_runtime                      -> '<unavailable>'
model_name                                -> 'sagemaker-xgboost-2024-06-24-05-19-43'
output_path                               -> 's3://housing-dataset-8749/housing/model'
predictor                                 -> '<unavailable>'
region                                    -> 'us-east-1'
role     

In [3]:
# get bucket_name
%store -r bucket_name
print(bucket_name)

housing-dataset-3320


In [4]:
# save Amazon information
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
s3 = boto3.client('s3', region_name=sagemaker_session.boto_region_name)
# Create the SageMaker Boto3 client
sm_client = boto3.client('sagemaker')

In [5]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

### Construct Athena queries to read the data from each offline feature store

In [6]:
train_feature_group_name = "housing_train"
validation_feature_group_name = "housing_validation"
test_feature_group_name = "housing_test"

In [7]:
sagemaker_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'housing_validation',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_validation',
   'CreationTime': datetime.datetime(2024, 6, 26, 5, 0, 33, 206000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_train',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_train',
   'CreationTime': datetime.datetime(2024, 6, 26, 5, 0, 11, 820000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_test',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_test',
   'CreationTime': datetime.datetime(2024, 6, 26, 5, 0, 54, 639000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}}],
 'ResponseMetadata': {'RequestId': '80049f9d-d42b-4

In [8]:
# Initialize FeatureGroup
train_feature_group = FeatureGroup(name=train_feature_group_name, sagemaker_session=sagemaker_session)

In [9]:
# Initialize FeatureGroup
validation_feature_group = FeatureGroup(name=validation_feature_group_name, sagemaker_session=sagemaker_session)

In [10]:
# Initialize FeatureGroup
test_feature_group = FeatureGroup(name=test_feature_group_name, sagemaker_session=sagemaker_session)

In [11]:
train_data_query = train_feature_group.athena_query()
validation_data_query = validation_feature_group.athena_query()
test_data_query = test_feature_group.athena_query()

In [12]:
train_data_table = train_data_query.table_name
validation_data_table = validation_data_query.table_name
test_data_table = test_data_query.table_name

In [13]:
train_query = f"""
SELECT * FROM "{train_data_table}"
"""

In [14]:
validation_query = f"""
SELECT * FROM "{validation_data_table}"
"""

In [15]:
test_query = f"""
SELECT * FROM "{test_data_table}"
"""

### Execute queries and put into dataframes

In [16]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_train = pd.DataFrame()
train_data_query.run(query_string=train_query, output_location='s3://'+bucket_name+'/query_results/train/')
train_data_query.wait()
df_train = train_data_query.as_dataframe()

In [17]:
# Check if the DataFrame is empty
if df_train.empty:
    print("DataFrame is empty. Trying to load data from S3...")
    
    # Run the query and wait for it to finish
    train_data_query.run(query_string=train_query, output_location='s3://'+bucket_name+'/query_results/train/')
    train_data_query.wait()
    
    # Pause for 5 seconds to ensure the data has time to load
    time.sleep(5)
    
    # Try to load the data into the DataFrame
    df_train = train_data_query.as_dataframe()

In [18]:
# Check if the DataFrame is still empty
if df_train.empty:
    print("No data in your S3 bucket for training. Confirm your data is there and rerun the notebook from the beginning.")
    raise SystemExit("Stopping script due to lack of data.")
else:
    print("Data successfully loaded into DataFrame.")

Data successfully loaded into DataFrame.


In [19]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_validation = pd.DataFrame()
validation_data_query.run(query_string=validation_query, output_location='s3://'+bucket_name+'/query_results/validation/')
validation_data_query.wait()
df_validation = validation_data_query.as_dataframe()

In [20]:
# Check if the DataFrame is empty
if df_validation.empty:
    print("DataFrame is empty. Trying to load data from S3...")
    
    # Run the query and wait for it to finish
    validation_data_query.run(query_string=validation_query, output_location='s3://'+bucket_name+'/query_results/validation/')
    validation_data_query.wait()
    
    # Pause for 5 seconds to ensure the data has time to load
    time.sleep(5)
    
    # Try to load the data into the DataFrame
    df_validation = validation_data_query.as_dataframe()

In [21]:
# Check if the DataFrame is still empty
if df_validation.empty:
    print("No data in your S3 bucket for validation. Confirm your data is there and rerun the notebook from the beginning.")
    raise SystemExit("Stopping script due to lack of data.")
else:
    print("Data successfully loaded into DataFrame.")

Data successfully loaded into DataFrame.


In [22]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_test = pd.DataFrame()
test_data_query.run(query_string=test_query, output_location='s3://'+bucket_name+'/query_results/test/')
test_data_query.wait()
time.sleep(5)
df_test = test_data_query.as_dataframe()

In [23]:
# Check if the DataFrame is empty
if df_test.empty:
    print("DataFrame is empty. Trying to load data from S3...")
    
    # Run the query and wait for it to finish
    test_data_query.run(query_string=test_query, output_location='s3://'+bucket_name+'/query_results/test/')
    test_data_query.wait()
    
    # Pause for 5 seconds to ensure the data has time to load
    time.sleep(5)
    
    # Try to load the data into the DataFrame
    df_test = test_data_query.as_dataframe()

In [24]:
# Check if the DataFrame is still empty
if df_test.empty:
    print("No data in your S3 bucket for testing. Confirm your data is there and rerun the notebook from the beginning.")
    raise SystemExit("Stopping script due to lack of data.")
else:
    print("Data successfully loaded into DataFrame.")

Data successfully loaded into DataFrame.


### Check the shape of data before dropping unnecessary columns and the saleprice column in the test dataframe

In [25]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(2043, 40)
(584, 40)
(292, 40)


### Get Y values for prediction inference and model quality

In [26]:
df_y_validation = df_validation['saleprice']
df_y_test = df_test['saleprice']

### For df_train and df_validation the "id" column is removed and for df_test both the "id" column and "saleprice" column are removed

In [27]:
df_train = df_train.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_validation = df_validation.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_test = df_test.drop(['id', 'saleprice', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)

### Check the shape of the data

In [28]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(2043, 35)
(584, 35)
(292, 34)


In [29]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mssubclass     2043 non-null   float64
 1   lotarea        2043 non-null   float64
 2   yearbuilt      2043 non-null   float64
 3   yearremodadd   2043 non-null   float64
 4   masvnrarea     2043 non-null   float64
 5   bsmtfinsf1     2043 non-null   float64
 6   bsmtfinsf2     2043 non-null   float64
 7   bsmtunfsf      2043 non-null   float64
 8   totalbsmtsf    2043 non-null   float64
 9   firstflrsf     2043 non-null   float64
 10  secondflrsf    2043 non-null   float64
 11  lowqualfinsf   2043 non-null   float64
 12  grlivarea      2043 non-null   float64
 13  bsmtfullbath   2043 non-null   float64
 14  bsmthalfbath   2043 non-null   float64
 15  fullbath       2043 non-null   float64
 16  halfbath       2043 non-null   float64
 17  bedroomabvgr   2043 non-null   float64
 18  kitchena

### Check the first few rows of the data

In [30]:
df_train.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,20.0,12299.0,1978.0,1985.0,67.0,1328.0,0.0,201.0,1529.0,1664.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2007.0,163000.0,12.001506
1,20.0,11310.0,1954.0,1954.0,0.0,0.0,0.0,1367.0,1367.0,1375.0,...,30.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0,140000.0,11.849398
2,20.0,14450.0,2006.0,2007.0,315.0,0.0,0.0,2121.0,2121.0,2121.0,...,98.0,0.0,0.0,142.0,0.0,0.0,5.0,2007.0,415298.0,12.936751
3,30.0,5142.0,1923.0,2008.0,0.0,224.0,0.0,725.0,949.0,949.0,...,0.0,183.0,0.0,0.0,0.0,0.0,6.0,2008.0,163000.0,12.001506
4,50.0,7000.0,1940.0,1950.0,0.0,375.0,0.0,345.0,720.0,720.0,...,0.0,30.0,0.0,0.0,0.0,0.0,5.0,2007.0,163000.0,12.001506


In [31]:
df_validation.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,60.0,10400.0,1980.0,1988.0,280.0,0.0,0.0,738.0,738.0,1277.0,...,73.0,0.0,0.0,0.0,0.0,0.0,3.0,2008.0,163000.0,12.001506
1,60.0,11050.0,1998.0,1999.0,0.0,615.0,0.0,434.0,1049.0,1036.0,...,70.0,0.0,0.0,0.0,0.0,0.0,5.0,2010.0,163000.0,12.001506
2,160.0,1680.0,1972.0,1972.0,297.0,0.0,0.0,483.0,483.0,483.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0,94500.0,11.456355
3,60.0,15600.0,1950.0,1963.0,405.0,760.0,0.0,408.0,1168.0,1278.0,...,0.0,0.0,0.0,192.0,0.0,0.0,7.0,2006.0,163000.0,12.001506
4,80.0,14115.0,1980.0,1980.0,225.0,1036.0,0.0,336.0,1372.0,1472.0,...,48.0,0.0,0.0,0.0,0.0,0.0,6.0,2009.0,187500.0,12.141534


In [32]:
df_test.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,logsaleprice
0,160.0,2117.0,2000.0,2000.0,513.0,420.0,0.0,336.0,756.0,756.0,...,0.0,32.0,0.0,0.0,0.0,0.0,0.0,6.0,2007.0,12.034691
1,20.0,8510.0,1971.0,1971.0,178.0,500.0,0.0,543.0,1043.0,1050.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2009.0,12.001506
2,160.0,1890.0,1976.0,1976.0,0.0,0.0,0.0,546.0,546.0,546.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2007.0,12.001506
3,20.0,12285.0,1960.0,1960.0,128.0,544.0,0.0,785.0,1329.0,1329.0,...,0.0,0.0,203.0,0.0,0.0,0.0,0.0,8.0,2009.0,12.001506
4,20.0,9350.0,1961.0,1995.0,217.0,310.0,512.0,491.0,1313.0,1313.0,...,172.0,28.0,0.0,0.0,121.0,0.0,0.0,2.0,2009.0,12.001506


In [33]:
df_train['saleprice'].head(2)

0    163000.0
1    140000.0
Name: saleprice, dtype: float64

In [34]:
df_validation['saleprice'].head(2)

0    163000.0
1    163000.0
Name: saleprice, dtype: float64

### Save the data to the S3 bucket as CSV files

In [35]:
df_train_x_with_header = df_train

In [36]:
df_train_x_with_header.to_csv(f"s3://{bucket_name}/processed/housing/train_x_with_header.csv", header=True,  index=False)

In [37]:
df_train.to_csv(f"s3://{bucket_name}/processed/housing/train.csv", header=False,  index=False)

In [38]:
df_train_saved_to_s3_path = 's3://{}/processed/housing/'.format(bucket_name)

In [39]:
df_validation_x_with_header = df_validation

In [40]:
df_validation_x_with_header.to_csv(f"s3://{bucket_name}/processed/housing/validation_x_with_header.csv", header=True, index=False)

In [41]:
df_validation.to_csv(f"s3://{bucket_name}/processed/housing/validation.csv", header=False, index=False)

In [42]:
df_validation_saved_to_s3_path = 's3://{}/processed/housing/'.format(bucket_name)

In [43]:
df_test_x_with_header = df_test

In [44]:
df_test_x_with_header.to_csv(f"s3://{bucket_name}/processed/housing/test_x_with_header.csv", header=True, index=False)

In [45]:
df_test.to_csv(f"s3://{bucket_name}/processed/housing/test.csv", header=False, index=False)

In [46]:
df_test_saved_to_s3_path = 's3://{}/processed/housing/'.format(bucket_name)

### Train a model using SageMaker built-in XgBoost algorithm on the training data and validate it on the validation data
1. set the container image for the XGBoost algorithm and estimator parameters
2. set the output path and model parameters
3. set the hyperparameters
4. create an estimator
5. fit the model to the training data and validate on the validation data

In [47]:
# Get the current date and time
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Update the model_name variable
model_name = f"sagemaker-xgboost-{current_datetime}"

# set the container image for the XGBoost algorithm
container = image_uris.retrieve(region=region, framework="xgboost", version="latest")

estimator_instance_count = 1

estimator_instance_type = "ml.m5.xlarge"

# Print the model name
print(model_name)

sagemaker-xgboost-2024-06-26-15-15-35


In [48]:
# set the output path
output_path = f"s3://{bucket_name}/housing/model"

# Location in S3 where the model checkpoint will be stored
model_checkpoint_s3_path = '{}/{}/checkpoint/'.format(output_path, model_name)

# Location in S3 where the trained model will be stored
model_output_s3_path = '{}'.format(output_path)

# Location where the trained model will be stored locally in the container before being uploaded to S3
model_local_dir = '/opt/ml/input/data/train'

In [49]:
# set the hyperparameters
hyperparameters = {
    "objective": "reg:linear",
    "num_round": "100",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "silent": "0",
}

In [50]:
# create an estimator
estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count= estimator_instance_count,
    instance_type= estimator_instance_type,
    checkpoint_s3_uri=model_checkpoint_s3_path,    
    output_path=output_path,
    hyperparameters=hyperparameters,
    model_dir=model_local_dir,
)

In [51]:
# fit the model to the training data and validate on the validation data
estimator.fit(
    {
        "train": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/train.csv", content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/validation.csv", content_type="text/csv"
        ),
    }
)

INFO:sagemaker:Creating training-job with name: xgboost-2024-06-26-15-15-36-091


2024-06-26 15:15:36 Starting - Starting the training job...
2024-06-26 15:15:58 Starting - Preparing the instances for training...
2024-06-26 15:16:26 Downloading - Downloading input data...
2024-06-26 15:16:50 Downloading - Downloading the training image...
2024-06-26 15:17:36 Training - Training image download completed. Training in progress.
2024-06-26 15:17:36 Uploading - Uploading generated training model.Arguments: train
[2024-06-26:15:17:29:INFO] Running standalone xgboost training.
[2024-06-26:15:17:29:INFO] File size need to be processed in the node: 0.49mb. Available memory size in the node: 8144.61mb
[2024-06-26:15:17:29:INFO] Determined delimiter of CSV input is ','
[15:17:29] S3DistributionType set as FullyReplicated
[15:17:29] 2043x34 matrix with 69462 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-06-26:15:17:29:INFO] Determined delimiter of CSV input is ','
[15:17:29] S3DistributionType set as FullyReplicated
[15:17:29] 584x34 m

### Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [52]:
# Saving training job information to be used in the ML lineage module
training_job_info = estimator.latest_training_job.describe()
if training_job_info != None:
    
    # Get the model data
    model_data = training_job_info["ModelArtifacts"]["S3ModelArtifacts"]
    # Create the primary container
    primary_container = {"Image": container, "ModelDataUrl": model_data}

    # Save our model to the Sagemaker Model Registry
    create_model_response = sagemaker_client.create_model(
        ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
    )
    print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:453322325373:model/sagemaker-xgboost-2024-06-26-15-15-35


In [53]:
# Inspect Training Job Details
training_job_info

{'TrainingJobName': 'xgboost-2024-06-26-15-15-36-091',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:453322325373:training-job/xgboost-2024-06-26-15-15-36-091',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://housing-dataset-3320/housing/model/xgboost-2024-06-26-15-15-36-091/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'reg:linear',
  'silent': '0',
  'subsample': '0.7'},
 'AlgorithmSpecification': {'TrainingImage': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'train:merror',
    'Regex': '.*\\[[0-9]+\\].*#011train-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:mae',
    'Regex':

### Deploy the model

#### A) Set the deployment parameters

1. Deployment instance details:

    1. Instance count
    
    2. Instance type
    
    3. The Elastic Inference accelerator type
    
2. Serializer and deserializer - set them to CSV as the data will be in CSV format.

In [54]:
# Set the instance count, instance type, endpoint name and other parameters
deploy_initial_instance_count = 1
deploy_instance_type = 'ml.m5.xlarge'
accelerator_type = None
endpoint_name = 'housing-endpoint'
serializer = CSVSerializer(content_type='text/csv')
deserializer = CSVDeserializer(encoding='utf-8', accept='text/csv')

#### B) Set the data capture parameters

Setting up data capture will automatically capture the specified request/response data for calls made to the SageMaker inference endpoint and store them in the specified location in S3.  This is required for the Model Monitor to work.

In [55]:
# Location in S3 that will contain all model monitor related files
model_monitor_s3_path = '{}/{}/model-monitor'.format(output_path, model_name)

# Location in S3 where the data captured from the endpoint will be stored.
# Do not add a '/' at the end of the path here; it will be automatically added by the endpoint. If you add
# a '/' at the end, then the monitoring schedule will not be able to find the data capture files and your
# schedule will fail even if files are present in that S3 location.
data_capture_output_s3_path = '{}/data-capture'.format(model_monitor_s3_path)

# Location in S3 where the results from the baseline job will be stored
data_quality_baseline_job_output_s3_path = '{}/data-quality/baseline/'.format(model_monitor_s3_path)

# Location in S3 where the results from the monitoring job will be stored
# Do not add a '/' at the end of the path here
data_quality_mon_schedule_output_s3_path = '{}/data-quality/monitoring'.format(model_monitor_s3_path)

In [56]:
# Set the data capture configuration
data_capture_config = DataCaptureConfig(
    enable_capture = True,
    sampling_percentage=100,
    destination_s3_uri=data_capture_output_s3_path,
    capture_options=['REQUEST', 'RESPONSE'],
    csv_content_types=['text/csv'],
    json_content_types=None,
    sagemaker_session=sagemaker.Session())

#### C.) Deploy the Model

In [57]:
# Deploy the model and automatically create the endpoint configuration in the process
predictor = estimator.deploy(initial_instance_count=deploy_initial_instance_count,
                             instance_type=deploy_instance_type,
                             accelerator_type=accelerator_type,
                             serializer=serializer,
                             deserializer=deserializer,
                             model_name=model_name,
                             endpoint_name=endpoint_name,
                             data_capture_config=data_capture_config,
                             wait=True)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-26-15-15-35
INFO:sagemaker:Creating endpoint-config with name housing-endpoint
INFO:sagemaker:Creating endpoint with name housing-endpoint


------!

### Create the Data Quality Monitor

#### A.) Set the Data Quality Monitor paramaters

In [58]:
# Specify parameters
data_quality_instance_count = 1
data_quality_instance_type = 'ml.m5.xlarge'
data_quality_instance_volume_size_in_gb = 5
data_quality_max_run_time_in_seconds = 1800
data_quality_base_job_name = 'data-quality-{}'.format(model_name)
data_quality_baseline_job_name = 'data-quality-bsl-job-{}'.format(model_name)
data_quality_monitor_schedule_name = 'data-quality-mon-sch-{}'.format(model_name)
data_quality_baseline_dataset_format = DatasetFormat.csv(header=True, output_columns_position='START')

# Location in S3 where the data captured from the endpoint will be stored.
data_quality_baseline_job_output_s3_path = '{}/data-quality/baseline/'.format(model_monitor_s3_path)

# Location in S3 where the results from the monitoring job will be stored
data_quality_mon_schedule_output_s3_path = '{}/data-quality/monitoring/'.format(model_monitor_s3_path)

model_quality_base_job_name = 'model-quality-{}'.format(model_name)
model_quality_baseline_job_name_prefix = 'model-quality-bsl-job-{}'.format(model_name)
model_quality_monitor_schedule_name = 'model-quality-mon-sch-{}'.format(model_name)


In [59]:
# Create the Data Quality Monitor
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=data_quality_instance_count,
    instance_type=data_quality_instance_type,
    volume_size_in_gb=data_quality_instance_volume_size_in_gb,
    max_runtime_in_seconds=data_quality_max_run_time_in_seconds,
    base_job_name=data_quality_base_job_name,
    sagemaker_session=sagemaker_session
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


#### B.) Generate the Baseline

In [60]:
train_x_with_header_in_S3 = 's3://{}/processed/housing/train_x_with_header.csv'.format(bucket_name)

In [61]:
# Create the baseline job and generate the constraints
data_quality_monitor.suggest_baseline(
    job_name=data_quality_baseline_job_name,
    baseline_dataset=train_x_with_header_in_S3,
    dataset_format=data_quality_baseline_dataset_format,
    output_s3_uri=data_quality_baseline_job_output_s3_path,
    wait=True,
    logs=True
)

INFO:sagemaker:Creating processing-job with name data-quality-bsl-job-sagemaker-xgboost-2024-06-26-15-15-35


............2024-06-26 15:23:52.573256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-26 15:23:52.573288: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-26 15:23:54.293639: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-26 15:23:54.293673: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-26 15:23:54.293695: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-217-12.ec2.internal): /proc/driver/nvidia/version does not exist
2024-06-26 15:23:54.293977: I tensorf

#### C.) Display the generated Data Quality Baseline

In [62]:
# Get the latest baselining job
data_quality_baseline_job = data_quality_monitor.latest_baselining_job

# Print the statistics
print('\nData Quality statistics:\n')
print(data_quality_baseline_job.baseline_statistics().body_dict['features'])

# Print the constraints
print('\nData Quality constraints:\n')
print(data_quality_baseline_job.suggested_constraints().body_dict['features'])

# Print the monitoring config
print('\nData Quality monitoring config:\n')
print(data_quality_baseline_job.suggested_constraints().body_dict['monitoring_config'])


Data Quality statistics:

[{'name': 'mssubclass', 'inferred_type': 'Fractional', 'numerical_statistics': {'common': {'num_present': 2043, 'num_missing': 0}, 'mean': 56.86735193343123, 'sum': 116180.0, 'std_dev': 41.73636492095891, 'min': 20.0, 'max': 190.0, 'distribution': {'kll': {'buckets': [{'lower_bound': 20.0, 'upper_bound': 37.0, 'count': 843.0}, {'lower_bound': 37.0, 'upper_bound': 54.0, 'count': 229.0}, {'lower_bound': 54.0, 'upper_bound': 71.0, 'count': 493.0}, {'lower_bound': 71.0, 'upper_bound': 88.0, 'count': 131.0}, {'lower_bound': 88.0, 'upper_bound': 105.0, 'count': 83.0}, {'lower_bound': 105.0, 'upper_bound': 122.0, 'count': 125.0}, {'lower_bound': 122.0, 'upper_bound': 139.0, 'count': 0.0}, {'lower_bound': 139.0, 'upper_bound': 156.0, 'count': 1.0}, {'lower_bound': 156.0, 'upper_bound': 173.0, 'count': 92.0}, {'lower_bound': 173.0, 'upper_bound': 190.0, 'count': 46.0}], 'sketch': {'parameters': {'c': 0.64, 'k': 2048.0}, 'data': [[20.0, 20.0, 20.0, 30.0, 50.0, 20.0, 20

### Schedule monitoring jobs

#### A.) Create Data Quality monitoring schedule

In [63]:
## Data Quality schedule

# Create the monitoring schedule
data_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_quality_monitor_schedule_name,
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=data_quality_mon_schedule_output_s3_path,
    statistics=data_quality_monitor.baseline_statistics(),
    constraints=data_quality_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35


#### B.) Print Schedule Details

In [64]:
## Data Quality details
# Describe and print status
data_quality_monitor_schedule_details = data_quality_monitor.describe_schedule()
print('\nData Quality Monitor - schedule details:\n')
print(data_quality_monitor_schedule_details)
print('\nData Quality Monitor - schedule status: {}'.format(data_quality_monitor_schedule_details['MonitoringScheduleStatus']))


Data Quality Monitor - schedule details:

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:453322325373:monitoring-schedule/data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35', 'MonitoringScheduleName': 'data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35', 'MonitoringScheduleStatus': 'Pending', 'MonitoringType': 'DataQuality', 'CreationTime': datetime.datetime(2024, 6, 26, 15, 27, 17, 867000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 6, 26, 15, 27, 17, 934000, tzinfo=tzlocal()), 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'}, 'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-15-27-17-243', 'MonitoringType': 'DataQuality'}, 'EndpointName': 'housing-endpoint', 'ResponseMetadata': {'RequestId': 'ecc108f7-d199-4756-bad4-de1e08f44a93', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ecc108f7-d199-4756-bad4-de1e08f44a93', 'content-type': 'application/x-amz-json-1.1', 'cont

#### C.) Start/stop/delete schedules

In [65]:
## Data Quality schedule controls
#dq_monitor.start_monitoring_schedule()
#dq_monitor.stop_monitoring_schedule()
#dq_monitor.delete_monitoring_schedule()

### Send traffic to endpoint 

In [66]:
# Set the inference id prefix to consistently use it when invoking the endpoint
# at various steps in this notebook
inference_id_prefix = 'FromNotebook_'

In [67]:
# Dataframe of the Test Dataset to a list
df_test_x_with_header_csv_data = df_test.values.tolist()

In [68]:
# Iterate over the test data and call the predictor for each row
for index, x_test_row in enumerate(df_test_x_with_header_csv_data, start=1):
    x_test_row_string = ','.join(map(str, x_test_row))
    # Auto-generate an inference-id to track the request/response in the captured data
    inference_id = '{}{}'.format(inference_id_prefix, index)
    predicted_object = predictor.predict(data=x_test_row_string,
                                         target_model=None,
                                         target_variant=None,
                                         inference_id=inference_id)
    predicted_value = float(predicted_object[0][0])
    print('{}: Prediction for \'{}\' is {}'.format(inference_id, x_test_row_string, predicted_value))

FromNotebook_1: Prediction for '160.0,2117.0,2000.0,2000.0,513.0,420.0,0.0,336.0,756.0,756.0,756.0,0.0,1512.0,0.0,0.0,2.0,1.0,2.0,1.0,4.0,1.0,2000.0,2.0,440.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,6.0,2007.0,12.034690856933594' is 193.82492065429688
FromNotebook_2: Prediction for '20.0,8510.0,1971.0,1971.0,178.0,500.0,0.0,543.0,1043.0,1050.0,0.0,0.0,1050.0,1.0,0.0,1.0,1.0,3.0,1.0,6.0,0.0,1971.0,1.0,336.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2009.0,12.001505851745604' is 187.6470489501953
FromNotebook_3: Prediction for '160.0,1890.0,1976.0,1976.0,0.0,0.0,0.0,546.0,546.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,6.0,0.0,1976.0,1.0,286.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2007.0,12.001505851745604' is 97.71820831298828
FromNotebook_4: Prediction for '20.0,12285.0,1960.0,1960.0,128.0,544.0,0.0,785.0,1329.0,1329.0,0.0,0.0,1329.0,0.0,0.0,1.0,1.0,3.0,1.0,5.0,2.0,1960.0,2.0,441.0,0.0,0.0,203.0,0.0,0.0,0.0,0.0,8.0,2009.0,12.001505851745604' is 180.5640411376953
FromNotebook_5: Prediction for '20.0,9350.0,1

### Analyze monitoring executions

#### A.) View monitoring executions

In [69]:
# List monitoring schedules for endpoint
print('Monitoring schedules for endpoint \'{}\':\n'.format(endpoint_name))
print(sm_client.list_monitoring_schedules(EndpointName=endpoint_name))

Monitoring schedules for endpoint 'housing-endpoint':

{'MonitoringScheduleSummaries': [{'MonitoringScheduleName': 'data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35', 'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:453322325373:monitoring-schedule/data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35', 'CreationTime': datetime.datetime(2024, 6, 26, 15, 27, 17, 867000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 6, 26, 15, 27, 17, 934000, tzinfo=tzlocal()), 'MonitoringScheduleStatus': 'Pending', 'EndpointName': 'housing-endpoint', 'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-15-27-17-243', 'MonitoringType': 'DataQuality'}], 'ResponseMetadata': {'RequestId': '5a2a215e-c07a-4524-8aa5-3bbe4c05b72f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5a2a215e-c07a-4524-8aa5-3bbe4c05b72f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '521', 'date': 'Wed, 26 Jun 2024 15:27:20 GMT'}, 'RetryAttempts': 

In [70]:
## Data Quality Monitor - monitoring executions
print('Monitoring executions for schedule \'{}\':\n'.format(data_quality_monitor_schedule_name))
print(sm_client.list_monitoring_executions(MonitoringScheduleName=data_quality_monitor_schedule_name,
                                           EndpointName=predictor.endpoint_name,
                                           MonitoringTypeEquals='DataQuality',
                                           SortBy='CreationTime',
                                           SortOrder='Descending'))

Monitoring executions for schedule 'data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35':

{'MonitoringExecutionSummaries': [], 'ResponseMetadata': {'RequestId': '12204352-825f-436e-9f34-ff5629917366', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '12204352-825f-436e-9f34-ff5629917366', 'content-type': 'application/x-amz-json-1.1', 'content-length': '35', 'date': 'Wed, 26 Jun 2024 15:27:20 GMT'}, 'RetryAttempts': 0}}


#### B.) View latest constraint violations

In [71]:
## Data Quality Monitor
data_quality_constraint_violations = data_quality_monitor.latest_monitoring_constraint_violations()
if data_quality_constraint_violations != None:
    print(data_quality_constraint_violations.body_dict)

### Cleanup

In [75]:
# Get the description of the monitoring schedule
response = sm_client.describe_monitoring_schedule(MonitoringScheduleName=data_quality_monitor_schedule_name)

# Get the status of the monitoring schedule
status = response['MonitoringScheduleStatus']

# Check if the status is 'Pending'
if status == 'Pending':
    print("The monitoring schedule is still pending.")
else:
    print(f"The monitoring schedule status is {status}. You can now delete the monitoring schedule.")
    # Delete the monitoring schedules
    data_quality_monitor.delete_monitoring_schedule()

INFO:sagemaker:Deleting Monitoring Schedule with name: data-quality-mon-sch-sagemaker-xgboost-2024-06-26-15-15-35


The monitoring schedule status is Scheduled. You can now delete the monitoring schedule.


INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2024-06-26-15-27-17-243


In [76]:
# Delete the model, endpoint configuration and endpoint
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: sagemaker-xgboost-2024-06-26-15-15-35
INFO:sagemaker:Deleting endpoint configuration with name: housing-endpoint
INFO:sagemaker:Deleting endpoint with name: housing-endpoint


In [79]:
# Delete all the data from the S3 buckets and delete the buckets
boto3_session = boto3.Session()
delete_S3_Buckets = boto3_session.resource('s3')
for bucket in delete_S3_Buckets.buckets.all():
    print(f'Deleting bucket {bucket.name}')
    
    # Delete all objects in the bucket
    bucket.objects.all().delete()
    
    # Delete the bucket itself
    bucket.delete()

print('All buckets have been deleted.')

All buckets have been deleted.


### Shut down notebook resources

In [80]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [81]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>